In [ ]:
import pandas as pd
import geopandas as gpd
from shapely import geometry
from tqdm import tqdm
import os
import tempfile
import sys

In [ ]:
# Add the src directory to the Python path
notebook_dir = os.getcwd()  # Current working directory of the notebook
parent_dir = os.path.abspath(os.path.join(notebook_dir, '..'))
if parent_dir not in sys.path:
    sys.path.append(parent_dir)

In [ ]:
from google.colab import drive

drive.mount('/content/drive')

In [ ]:
AOI_PATH = '/content/drive/Shareddrives/Sunbird AI/Projects/GIZ Mini-grid Identification/Phase II/Data/administrative areas/UGA Lamwo district.gpkg'
lamwo = gpd.read_file(AOI_PATH)

In [ ]:
lamwo.geometry.iloc[0]

In [ ]:
# Geometry copied from drive
"""aoi_geom = {
    "coordinates": [
        [
            [-122.16484503187519, 47.69090474454916],
            [-122.16484503187519, 47.6217555345674],
            [-122.06529607517405, 47.6217555345674],
            [-122.06529607517405, 47.69090474454916],
            [-122.16484503187519, 47.69090474454916],
        ]
    ],
    "type": "Polygon",
}"""
aoi_shape = lamwo.geometry.iloc[0]
minx, miny, maxx, maxy = aoi_shape.bounds
print(aoi_shape.bounds)

output_fn = "lamwo_building_footprints.geojson"

In [ ]:
aglwb = 'https://services2.arcgis.com/g8WusZB13b9OegfU/arcgis/rest/services/Aboveground_Live_Woody_Biomass_Density/FeatureServer/0/query?where=1%3D1&outFields=tile_id,Mg_px_1_download,Shape__Area,Shape__Length&outSR=4326&f=json'

In [ ]:
gdf = gpd.read_file(aglwb)
gdf.head()

In [ ]:
intersecting_tiles_gdf = gdf[gdf.intersects(lamwo.geometry.iloc[0])]

In [ ]:
import requests
import rasterio

# Create a directory to save the downloaded rasters
output_raster_dir = '/content/downloaded_rasters'
os.makedirs(output_raster_dir, exist_ok=True)

# Iterate through the intersecting tiles and download the rasters
downloaded_raster_paths = []
for index, row in intersecting_tiles_gdf.iterrows():
    url = row["Mg_px_1_download"]
    tile_id = row["tile_id"]
    output_path = os.path.join(output_raster_dir, f"{tile_id}.tif")

    print(f"Downloading {url} to {output_path}")
    try:
        # Download the raster file
        response = requests.get(url, stream=True)
        response.raise_for_status()  # Raise an HTTPError for bad responses (4xx or 5xx)

        with open(output_path, 'wb') as f:
            for chunk in response.iter_content(chunk_size=8192):
                f.write(chunk)

        downloaded_raster_paths.append(output_path)
        print(f"Successfully downloaded {tile_id}.tif")

    except requests.exceptions.RequestException as e:
        print(f"Error downloading {url}: {e}")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")

print(f"\nDownloaded {len(downloaded_raster_paths)} raster files to {output_raster_dir}")

In [ ]:
from rasterio.mask import mask

# Create a directory to save the clipped rasters
output_clipped_raster_dir = '/content/clipped_rasters'
os.makedirs(output_clipped_raster_dir, exist_ok=True)

clipped_raster_paths = []

# Iterate through the downloaded rasters and clip them
for raster_path in downloaded_raster_paths:
    with rasterio.open(raster_path) as src:
        out_image, out_transform = mask(src, [lamwo.geometry.iloc[0]], crop=True)
        out_meta = src.meta.copy()

        out_meta.update({
            "driver": "GTiff",
            "height": out_image.shape[1],
            "width": out_image.shape[2],
            "transform": out_transform
        })

        # Construct the output path for the clipped raster
        clipped_output_path = os.path.join(output_clipped_raster_dir, os.path.basename(raster_path))

        with rasterio.open(clipped_output_path, "w", **out_meta) as dest:
            dest.write(out_image)

        clipped_raster_paths.append(clipped_output_path)
        print(f"Clipped raster saved to {clipped_output_path}")

print(f"\nClipped {len(clipped_raster_paths)} raster files to {output_clipped_raster_dir}")